In [174]:
import numpy as np
from PIL import Image
import re
from src.css.css_parser import *
from fpdf import FPDF
from src.css.color_utils import *
import os.path
from src.fpdf_extention import *
from src.md_to_pdf_converter import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
colors = get_theme_from_file("Obsidian Nord.css")

In [176]:
pool = StylePool()
pool.doc_style.set_background_color(colors["--background-primary"])
pool.style_normal.set_font_color(colors["--text-normal"])

pool.style_h1.set_font_color(colors["--text-title-h1"])
pool.style_h2.set_font_color(colors["--text-title-h2"])
pool.style_h3.set_font_color(colors["--text-title-h3"])
pool.style_h4.set_font_color(colors["--text-title-h4"])
pool.style_h5.set_font_color(colors["--text-title-h5"])
pool.style_h6.set_font_color(colors["--text-title-h6"])

pool.style_code.set_font_color(colors["--code-block"])
pool.style_code.set_background_color(colors["--pre-code"])

pool.style_mark.set_font_color(colors["--text-normal"])
pool.style_mark.set_background_color(colors["--pre-code"])

pool.style_link.set_font_color(colors["--text-accent"])


In [205]:
file_text = read_contents("MDtest.md")

In [178]:
pdf = PrintablePDF()
convert_md_to_pdf(file_text, pdf, pool)
#pdf.link(10,10,100,100, 'http|://www.intranet.com/mb/rprh06/final.php?folio=')

pdf.output("mygfg.pdf")

''

In [179]:
text = "[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF)"

regex = re.compile(r"\[(.*)\]\((.*)\)")
matches = regex.findall(text)

matches

[('link to the page', 'https://github.com/Altair200333/ObsidianMDtoPDF')]

In [209]:
pdf = PrintablePDF()
pdf.set_doc_style(pool.doc_style)

pdf.add_page()
pdf.set_text_style(pool.style_normal)

blocks = [H1Block(pool), H2Block(pool), H3Block(pool), H4Block(pool), H5Block(pool), H6Block(pool), MarkBlock(pool), CodeBlock(pool), LinkBlock(pool)]

def mark_block(block, id, table, string):
    results = [(m.start(0), m.end(0)) for m in block.regex.finditer(string)]
    for res in results:
        table[res[0]] = id
        table[res[1]] = -id
        print(string[res[0]:res[1]])

style_dict = {}

for i, block in enumerate(blocks):
    mark_block(block, i + 1, style_dict, file_text)

print(style_dict)

style_stack = []
current_style = None
for i, c in enumerate(file_text):
    if i in style_dict:
        value = style_dict[i]
        if value > 0:
            print("start style", abs(value))
            style_stack.append(value)
            current_style = blocks[abs(value) - 1]
            current_style.feed(c)

        if value < 0:
            print("end style", abs(value))
            style_stack.pop()
            current_style.feed(c)
            current_style.flush(pdf)
            current_style = None

            if c == '\n':
                pdf.print_char(c)
            
            # if there is was style before this make it the main one
            if len(style_stack) > 0:
                current_style = blocks[-1]
            
        pdf.finish_print()
        print(style_stack)
    else:
        # no style - normal text
        if current_style == None:
            pdf.print_char(c)
            
        else:
            current_style.feed(c)

    
pdf.finish_print()
pdf.output("mygfg.pdf")

# Title 1
## Title 2
### Obsidian2Pdf 
### Highlighted text l a w e t y u q we w rwq eqr ew
### Code blocks
### Different header styles
### Title 3
#### Title 4
##### Title 5
###### Title 6
>highlited text
>	more text 
```title
bottom text
```
[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF2)
{296: 1, 305: -1, 306: 2, 316: -2, 0: 3, 17: -3, 71: 3, 123: -3, 156: 3, 171: -3, 199: 3, 226: -3, 317: 3, 328: -3, 329: 4, 341: -4, 342: 5, 355: -5, 356: 6, 370: -6, 125: 7, 140: -7, 141: 7, 153: -7, 173: 8, 197: -8, 227: 9, 295: -9}
start style 3
[3]
end style 3
['Obsidian2Pdf ']
[]
start style 3
[3]
end style 3
['Highlighted text l a w e t y u q we w rwq eqr ew']
[]
start style 7
[7]
end style 7
['highlited text']
[]
start style 7
[7]
end style 7
['\tmore text ']
[]
start style 3
[3]
end style 3
['Code blocks']
[]
start style 8
[8]
end style 8
[]
start style 3
[3]
end style 3
['Different header styles']
[]
start style 9
[9]
end style 9
[]
start style 1
[1]
end style 1
['Title

''

In [181]:
results = [(m.start(0), m.end(0)) for m in H2Block(pool).regex.finditer(file_text)]
results

[(10, 20)]

In [163]:
res = re.findall(r"^## .*", "\n## Title 2",flags=re.MULTILINE)
res

['## Title 2']